In [164]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import datetime as dt
import yfinance as yfin
import scipy.optimize as sc
import plotly.graph_objects as go

yfin.pdr_override()

# Obtain Stock Price Data 

In [165]:
def getData(stocks,start,end):
    stockData = pdr.get_data_yahoo(stocks, start = start, end = end)
    stockData = stockData["Close"]
    
    return stockData.pct_change()
    

In [166]:
stock_list = ("TSLA", "GOOGL", "AAPL")
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 365)

# Percent Change of Closing Price form the last 365 days:

In [167]:
pctChange = getData(stock_list,startDate, endDate)

[*********************100%***********************]  3 of 3 completed


In [168]:
mean_returns = pctChange.mean()
cov_matrix = pctChange.cov()

## Mean Returns:

In [169]:
mean_returns

AAPL     0.000891
GOOGL    0.000225
TSLA    -0.000667
dtype: float64

In [170]:
cov_matrix

,AAPL,GOOGL,TSLA
AAPL,0.000429,0.000375,0.000490
GOOGL,0.000375,0.000583,0.000456
TSLA,0.000490,0.000456,0.001645


In [171]:
def portfolio_performance(weights,mean_returns, cov_matrix):
    returns = np.sum(mean_returns* weights) * 252 ## 252 trading days times mean returns into the weight of stock in a portfolio
    std = np.sqrt(np.dot(weights.T,np.dot(cov_matrix,weights))) * np.sqrt(252) # Portfolio Variance wT . Cov(Matrix) . w
    ## Multiply by sqrt(252) because of the volatitility rool where we need to annualise the std_dev of returns
    
    return returns,std

    

In [172]:
weights = np.array([0.3,0.3,0.4])

In [173]:
returns , std = portfolio_performance(weights,mean_returns,cov_matrix)

In [174]:
returns = round(returns*100,2)
std = round(std*100,2)

In [175]:
returns,std

(1.71, 40.44)

# Optimizing for Highest Sharpe Ratio

In [176]:
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate = 0):
    p_returns , p_std = portfolio_performance(weights,mean_returns,cov_matrix)
    
    return - (p_returns - risk_free_rate)/p_std

In [177]:
def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate = 0,constraint_set = (0,1)):
    
    #Max Sharpe Ratio = Min Negative Sharpe Ratio
    
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type':'eq','fun': lambda x:np.sum(x) - 1})
    bound = constraint_set
    bounds = tuple(bound for asset in range(num_assets))
    
    result =  sc.minimize(neg_sharpe_ratio, num_assets*[1.0/num_assets], args = args, method = "SLSQP", bounds = bounds, constraints = constraints)
    
    return result
    

In [178]:
out = max_sharpe_ratio(mean_returns, cov_matrix)
out



     fun: -0.6831188076201874
     jac: array([-7.45058060e-09,  4.25317660e-01,  1.29215978e+00])
 message: 'Optimization terminated successfully'
    nfev: 12
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([1.00000000e+00, 5.55111512e-16, 0.00000000e+00])

In [179]:
max_SR_ratio = out['fun']
maxSR_weights = out['x']

max_sharpe_ratio, maxSR_weights

(<function __main__.max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate=0, constraint_set=(0, 1))>,
 array([1.00000000e+00, 5.55111512e-16, 0.00000000e+00]))

# LOL Put all weights in Tesla

In [180]:
def portfolio_variance(weights, mean_returns, cov_matrix):
    return portfolio_performance(weights, mean_returns, cov_matrix)[1]

In [181]:
def minimize_variance(mean_returns, cov_matrix, risk_free_rate = 0,constraint_set = (0,1)):
    #We want to minimize the portfolio variance by changing the weights of the stocks in the portfolio
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type':'eq','fun': lambda x:np.sum(x) - 1})
    bound = constraint_set
    bounds = tuple(bound for asset in range(num_assets))
    
    result =  sc.minimize(portfolio_variance, num_assets*[1.0/num_assets], args = args, method = "SLSQP", bounds = bounds, constraints = constraints)
    
    return result

In [182]:
min_var_output = minimize_variance(mean_returns, cov_matrix)
min_var, min_var_weights = min_var_output['fun'], min_var_output['x']

min_var, min_var_weights

(0.3244418954605609, array([7.94965410e-01, 2.05034590e-01, 2.36356074e-17]))

# GOAL : 1) Maximize Sharpe Ration ; 2) Minimize Portfolio Volatility 

In [183]:
def calculated_results(mean_returns, cov_matrix, risk_free_rate = 0, constraint_set =(0,1)):

    
    # Want to read in the mean, cov_matrix, and other reqd info
    #Output the max sharpe ratio and the minimum volatitlity portfolio
    #And then the efficient frontier
    
     ## GOAL 1: MAX SHARPE RATIO PORTFOLIO FUNCTION
    
    
    max_SR_Portfolio = max_sharpe_ratio(mean_returns,cov_matrix)
    
    max_SR_weights = max_SR_Portfolio["x"]
    
    max_SR_returns, max_SR_std = portfolio_performance(max_SR_weights,mean_returns,cov_matrix)
    
    max_SR_allocation = pd.DataFrame(max_SR_Portfolio["x"],columns = ["allocations"], index = mean_returns.index)
    
    
    
    
     ## GOAL 2: MINIMIZE PORTFOLIO VARIANCE FUNCTION
        
    min_Vol_Portfolio = minimize_variance(mean_returns,cov_matrix)
    
    min_Vol_weights = min_Vol_Portfolio["x"]
    
    min_Vol_returns, min_Vol_std = portfolio_performance(min_Vol_weights,mean_returns,cov_matrix)
    
    min_Vol_allocation = pd.DataFrame(min_Vol_weights,columns = ["allocations"], index = mean_returns.index)
        
        
    ###VALUE OF TARGETS ON EFFICIENT FRONTIER:
    
    efficient_list = []
    
    target_returns = np.linspace(min_Vol_returns, max_SR_returns,20)
    
    for target in target_returns:
        efficient_list.append(efficient_optimizer(mean_returns, cov_matrix,target)['fun'])
        
    
    
    return max_SR_returns,max_SR_std,max_SR_allocation,min_Vol_returns,min_Vol_std,min_Vol_allocation,efficient_list,target_returns

In [184]:
max_SR_allocation = calculated_results(mean_returns, cov_matrix, risk_free_rate = 0, constraint_set =0)[2]

In [185]:
max_SR_allocation.allocations = [round(i*100,0) for i in max_SR_allocation.allocations]

In [186]:
min_Vol_allocation = calculated_results(mean_returns, cov_matrix, risk_free_rate = 0, constraint_set =0)[5]

In [187]:
min_Vol_allocation.allocations = [round(i*100,0) for i in min_Vol_allocation.allocations]

In [188]:
max_SR_allocation

,allocations
AAPL,100.0
GOOGL,0.0
TSLA,0.0


In [189]:
min_Vol_allocation

,allocations
AAPL,79.0
GOOGL,21.0
TSLA,0.0


# Creating Efficient Frontier

In [190]:
def efficient_optimizer(mean_returns, cov_matrix,return_target,constraint_set = (0,1)):
    
    ### FOR A GIVEN TARGET OF RETURNS WE WANT TO OPTIMISE THE PORTFOLIO FOR MIN VARIANCE AND MAX SHARPE RATIO
    
    ## HELPER FUNCTION FOR efficient_optimizer():

    def portfolio_returns(weights, mean_returns, cov_matrix):
        return portfolio_performance(weights, mean_returns, cov_matrix)[0]
    
    
    num_assets = len(mean_returns)
    args_ = (mean_returns,cov_matrix)
    
    constraints = ({'type':'eq','fun': lambda x: portfolio_returns(x,mean_returns,cov_matrix) - return_target}, #portfolio_return(x) is a helper function
                   {'type':'eq','fun': lambda x:np.sum(x) - 1})
    bound = constraint_set
    
    bounds = tuple(bound for asset in range(num_assets))
    
    eff_optimizer = sc.minimize(portfolio_variance, num_assets*[1.0/num_assets], args = args_, method = "SLSQP", bounds = bounds, constraints = constraints)
    
    
    return eff_optimizer
    

In [191]:
efficient_optimizer(mean_returns, cov_matrix,0.05,constraint_set = (0,1))

     fun: 0.3635222592764618
     jac: array([0.28136021, 0.36040587, 0.46615634])
 message: 'Optimization terminated successfully'
    nfev: 20
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.19647795, 0.62719076, 0.17633129])

In [192]:
efficient_list = calculated_results(mean_returns, cov_matrix, risk_free_rate = 0, constraint_set =(0,1))[6]

In [193]:
efficient_list

[0.32444189546056107,
 0.32445372362694325,
 0.3244892371748438,
 0.3245484283776414,
 0.32463128423498294,
 0.32473778660864777,
 0.32486791236389456,
 0.3250216329489091,
 0.32519891511458704,
 0.3253997201081703,
 0.32562400460573976,
 0.3258717201515257,
 0.3261428130892841,
 0.32643722554813626,
 0.3267548940911416,
 0.3270957513955566,
 0.32745972458249434,
 0.327846737024118,
 0.32825670725512707,
 0.328689548879345]

In [226]:
def efficient_frontier_graph(mean_returns, cov_matrix, risk_free_rate = 0, constraint_set = (0,1)):
     #OUTPUTS A GRAPH PLOTTING MAX SHARPE RATIO , MIN VOLATILITY AND EFFICIENT FRONTIER
        
        max_SR_returns,max_SR_std,max_SR_allocation,min_Vol_returns,min_Vol_std,min_Vol_allocation,efficient_list,target_returns = calculated_results(mean_returns, cov_matrix, risk_free_rate = 0, constraint_set =(0,1)) 
        
        # MAX SHARPE RATIO PLOTS
        max_SRs = go.Scatter(name = "Max Sharpe Ratios",
                             mode = 'markers',
                            x = [max_SR_std],
                            y = [max_SR_returns],
                            marker = dict(color = 'red', line = dict(width = 3, color = 'black') ))
        
        # MIN VOLATILITY PLOTS
        min_Vols = go.Scatter(name = "Min Volatility",
                             mode = 'markers',
                            x = [min_Vol_std],
                            y = [min_Vol_returns],
                            marker = dict(color = 'green',  line = dict(width = 3, color = 'black') ))
        
        # EFFICIENT FRONTIER CURVE
        eff_curve = go.Scatter(name = "Efficient Frontier",
                             mode = "lines",
                            x = [round(eff_std*100,2)for eff_std in efficient_list],
                            y = [round(target*100,2)for target in target_returns],
                            dash = dict(color = 'black', line = dict(width = 3, dash = "dashdot") ))
        
        data = [max_SRs,min_Vols,eff_curve]
        
        layout = go.Layout(title = "Portfolio Optimization with the Efficient Frontier",
                           yaxis = dict(title ="Annualised Return (%)"),
                           xaxis = dict(title ="Annualised Volatility (%)")  
                          )
        
        figure = go.Figure(data = data, layout = layout)
        
        return figure.show()
        
        

In [227]:
efficient_frontier_graph(mean_returns, cov_matrix)

ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'dash'

Did you mean "dx"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        dx
            Sets the x coordinate step. 

     fun: 0.3635222592764618
     jac: array([0.28136021, 0.36040587, 0.46615634])
 message: 'Optimization terminated successfully'
    nfev: 20
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.19647795, 0.62719076, 0.17633129])